#1. Prepare the environment

In [1]:
!nvidia-smi -L

GPU 0: Tesla K80 (UUID: GPU-800283d4-b651-fe9f-4d57-5823f42989c4)


In [2]:
import sys
print(f"Python {sys.version}\n")

import numpy as np
print(f"Numpy {np.__version__}\n")

import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
print(f"TensorFlow {tf.__version__}")
print(f"tf.keras.backend.image_data_format() = {tf.keras.backend.image_data_format()}")

# Count the number of GPUs as detected by tensorflow
gpus = tf.config.list_physical_devices('GPU')
print(f"TensorFlow detected {len(gpus)} GPU(s):")
for i,gpu in enumerate(gpus):
  print(f"...GPU No.{i} : Name = {gpu.name}, Type = {gpu.device_type}")

Python 3.7.12 (default, Jan 15 2022, 18:48:18) 
[GCC 7.5.0]

Numpy 1.21.5

TensorFlow 2.8.0
tf.keras.backend.image_data_format() = channels_last
TensorFlow detected 1 GPU(s):
...GPU No.0 : Name = /physical_device:GPU:0, Type = GPU


In [3]:
import glob
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
import cv2 as cv

#2. Prepair Image list

In [4]:
from google.colab import drive

drive.mount('/content/gdrive')

MessageError: ignored

In [ ]:
Sotorns = glob.glob('/content/gdrive/MyDrive/Colab Notebooks/Deep Learning/Image_Dataset/Sothon/*.png')  # 0-99
Tos = glob.glob('/content/gdrive/MyDrive/Colab Notebooks/Deep Learning/Image_Dataset/Toh/*.png') # 105-204
Banlambs = glob.glob('/content/gdrive/MyDrive/Colab Notebooks/Deep Learning/Image_Dataset/Wat Ban Laem/*.png') # 205-305
Watraikings = glob.glob('/content/gdrive/MyDrive/Colab Notebooks/Deep Learning/Image_Dataset/Wat Rai Khing/*.png') # 306-405
Thongs = glob.glob('/content/gdrive/MyDrive/Colab Notebooks/Deep Learning/Image_Dataset/Thong/*.png') # 406-505

In [ ]:
filelist = []
filelist.extend(Sotorns)
filelist.extend(Tos)
filelist.extend(Banlambs)
filelist.extend(Watraikings)
filelist.extend(Thongs)

In [ ]:
instances = []
img_w, img_h = 224,224
for file_name in filelist:
    img = tf.keras.preprocessing.image.load_img(file_name, target_size = (img_w, img_h))
    instances.append(img)

In [ ]:
x = tf.keras.preprocessing.image.img_to_array(instances[0])
plt.imshow(x.astype(np.uint8))

In [ ]:
#COnvert PIL image to Numpy Array
img_list = []

for img in instances:
    x = tf.keras.preprocessing.image.img_to_array(img)
    img_list.append(x.astype(np.uint8))

In [ ]:
img_array = np.array(img_list)
img_array.shape

In [ ]:
# get Y 
ans_list = ['Sothon','Toh','Wat Ban Laem','Wat Rai Khing','Thong']
name_index = []
for file in filelist:
  ans = file.split('/')[7]
  name_index.append(ans_list.index(ans))
print(name_index)

In [ ]:
name_index = np.expand_dims(name_index, axis = 1)

name_index.shape

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(img_array, name_index, test_size=0.33, random_state=42)

In [ ]:
#visualize

plt.figure(figsize = (15,5))
for i in range(5):
    plt.subplot(150+1+i).set_title(f"class no.{y_train[i]}: {ans_list[i]}")
    plt.imshow(x_train[i])
plt.show()

In [ ]:
# Plot y_train and count value
unique, counts = np.unique(y_train, return_counts=True)
dict(zip(unique, counts))

#2. Load the pretrained model

In [ ]:
# Load the ImageNet VGG-16 model, including both the feature extractor part and the classifier part
# By default, VGG-16 is designed for an input image of dimension 224x224x3

np.random.seed(1234)
tf.random.set_seed(5678)



base_model = tf.keras.applications.inception_v3.InceptionV3(weights="imagenet", include_top=False, input_shape=(img_w,img_h,3))
base_model.trainable = True # trainable weights

base_model.summary()

#3. Train Model

In [ ]:
#Preprocessing
x_train_preprocess = tf.keras.applications.inception_v3.preprocess_input(x_train)
x_test_preprocess = tf.keras.applications.inception_v3.preprocess_input(x_test)

In [ ]:
#Add new classification head

x = base_model.output

#add custom layer to end of existing model
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(512, activation = 'relu')(x)
x = tf.keras.layers.Dense(256, activation = 'relu')(x)
x = tf.keras.layers.Dense(128, activation = 'relu')(x)
x = tf.keras.layers.Dense(64, activation = 'relu')(x)
#x = tf.keras.layers.Dense(32, activation = 'relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
new_outputs = tf.keras.layers.Dense(5)(x)
new_outputs = tf.keras.activations.softmax(new_outputs)

#Constrauct the main model
model = tf.keras.models.Model(inputs = base_model.inputs, outputs = new_outputs)
model.compile(loss = "sparse_categorical_crossentropy", optimizer ="adam", metrics = ["accuracy"])
model.summary()

In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath="best_weights.hdf5",  monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

callbacks_list = [checkpoint]
#history = model.fit(x_train_vgg, y_train,
#          validation_data=(x_test_vgg,y_test),
#          callbacks=[callbacks_list],
#          batch_size=100,
#          verbose=2, epochs=100)

history = model.fit(x_train_preprocess, y_train, callbacks=[callbacks_list], batch_size=100, verbose=2, epochs=100, validation_split = 0.2)

In [ ]:
# Summarize history for accuracy
plt.figure(figsize=(15,5))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.grid()
plt.show()

# Summarize history for loss
plt.figure(figsize=(15,5))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.grid()
plt.show()

#5. Model Load

In [ ]:
model.load_weights('best_weights.hdf5')

In [ ]:
#Evaluation

results = model.evaluate(x_test_preprocess, y_test, batch_size = 100)
print(f"{model.metrics_names}: {results}")

In [ ]:
model.summary()

#6. Adapt the base model

In [ ]:
model.layers[-2].activation

In [ ]:
def alter_model_for_GradCAM( model, last_conv_layer_name ):
  # 1. Get the output from the layer
  last_conv_output = model.get_layer( last_conv_layer_name ).output
  out_linear = model.layers[-2].output
  out_softmax = model.layers[-1].output
  # 3. Create the new model and assign the weight for the 'prediction_linear' layer
  new_model = tf.keras.Model( inputs=model.inputs, outputs=[out_softmax, out_linear, last_conv_output] )

  return new_model

In [ ]:
new_model = alter_model_for_GradCAM( model, 'mixed10')
new_model.summary()

In [ ]:
new_model.output

#6. GradCAM

In [ ]:
# Load a test image from Internet
img_url =  "https://cv.lnwfile.com/rdk4q7.jpg"
#img_url = "http://img-187.uamulet.com/taradpra.com/UserProfile/SMaeKlong/Picture/Pic_1275472_5.jpg"
!wget "$img_url"

# Load an image as PIL.Image.IMage
img_path = img_url.split("/")[-1]
img = tf.keras.preprocessing.image.load_img(img_path, target_size=(img_w, img_h))

# Convert PIL Image to Numpy ndarray
img = tf.keras.preprocessing.image.img_to_array(img)
img = img.astype(np.uint8)

# Display the NumPy image
plt.imshow(img)
plt.show()

In [ ]:
# Preprocess the input image
img_preprocess = tf.keras.applications.vgg16.preprocess_input( np.expand_dims(img, axis=0))

In [ ]:
result = model.predict(img_preprocess)
print(result)

In [ ]:
def my_CNN_GradCAM( model, in_img, class_index ):
  # Convert from numpy to tensor
  in_img = tf.cast( in_img, tf.float32 )

  # Compute the gradient of y_c with respect to A_i
  with tf.GradientTape() as tape:
    tape.watch(in_img)
    y_softmax, y_linear, last_conv_activation = model( in_img )
    one_class_score = y_linear[..., class_index ]
    gradient = tape.gradient( one_class_score, last_conv_activation )

    # Compute a vector of alphas (one alpha value is one weight for one 2D feature map) by averaging each 2Dgradient
    gradient = gradient.numpy().squeeze(axis=0)
    alpha = np.mean( gradient, axis=(0,1))

    # Compute the weighted-sum heatmap
    last_conv_activation = last_conv_activation.numpy().squeeze(axis=0)
    heatmap = np.dot(last_conv_activation, alpha)

    # Don't hightlight any pixels with negative gradients (= apply ReLU)
    heatmap = np.maximum(0, heatmap)

    # Return an unnormalized heatmap
    return heatmap

In [ ]:
# Get predictions with highest scores
top = 5
#_, top_results = my_imagenet_decode_predictions( out_softmax, top=top )

# Extract 0-based class indices
#top_index = [ tp[0] for tp in top_results[0] ]
top_index = [0,1,2,3,4,5]
# Visualize Grad-CAM for each class whose index is in 'top_index'
#for i,c in enumerate(top_index):
#class_desc = f"{imagenet_dict[c][-1]}, Top {i+1} score of {out_softmax[...,c].squeeze()}"
for c in range(top):
  heatmap = my_CNN_GradCAM( new_model,       # the CNN model
                            img_preprocess,   # input of the model
                            c                 # the 0-based index of the output class to compute and visualize Grad-CAM
                            )
  # Create the main figure
  fig = plt.figure(figsize=(15,5))
  class_desc = ans_list[c]
  fig.suptitle(f"class no.{c} ({class_desc}) score : {result.item(c)}")


  # Visualize the original heatmap
  ax = fig.add_subplot(1,2,1)
  ax.set_title("Original Grad-CAM")
  im = ax.imshow(heatmap, cmap='jet')
  plt.colorbar(im,ax=ax)

  # Visualize the heatmap overlaid on an input image
  ax = fig.add_subplot(1,2,2)
  ax.set_title("Resized Grad-CAM on an input image")
  ax.imshow(img)
  alpha = 0.6
  im = ax.imshow(cv.resize(heatmap,img.shape[:2]), cmap='jet', alpha=alpha)
  plt.colorbar(im,ax=ax)

  #Show this figure (containing two subplots)
  plt.show()